In [1]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from scipy.ndimage import label, generate_binary_structure

# --- CONFIGURACIÓN (RUTAS ABSOLUTAS) ---

# 1. Ruta exacta donde están tus archivos PREDICHOS (.nii.gz)
pred_folder = r"C:\Users\marta\Desktop\Predicciones2D3D_Weighted7030_Threshold03\Predicciones"

# 2. Ruta exacta donde están tus ETIQUETAS REALES (Ground Truth)
gt_folder = r"C:\Users\marta\Desktop\Predicciones2D3D_Weighted7030_Threshold03\GT" 

# 3. Ruta donde quieres guardar el CSV
output_csv = os.path.join(pred_folder, "Resultados_Predicciones2D3D_Weighted7030_Threshold03.csv")


def get_cohort(case_id):
    try:
        # Asume formato "sub-XXX". Si tus archivos son diferentes, ajusta esto.
        num_part = case_id.split('-')[1]
        return int(num_part[0])
    except:
        return 0

def compute_metrics(gt_path, pred_path, case_id):
    # Cargar GT
    gt_img = nib.load(gt_path)
    gt_data = gt_img.get_fdata().astype(np.uint8)
    gt_data = (gt_data > 0).astype(np.uint8) # Binarizar

    # Cargar Predicción (si existe)
    if os.path.exists(pred_path):
        pred_img = nib.load(pred_path)
        pred_data = pred_img.get_fdata().astype(np.uint8)
        pred_data = (pred_data > 0).astype(np.uint8) # Binarizar
    else:
        print(f"Aviso: No hay predicción para {case_id}, se asume vacío.")
        pred_data = np.zeros_like(gt_data)

    # --- 1. Métrica Voxel-wise (Dice) ---
    intersection = np.sum(gt_data * pred_data)
    sum_gt = np.sum(gt_data)
    sum_pred = np.sum(pred_data)
    
    if sum_gt + sum_pred == 0:
        dice_voxel = 1.0 
    else:
        dice_voxel = (2. * intersection) / (sum_gt + sum_pred)

    # --- 2. Detección de Lesiones (Object-wise) ---
    
    # Mantenemos Conectividad (3, 2) como pediste
    structure = generate_binary_structure(3, 2)
    
    gt_labels, n_lesions_gt = label(gt_data, structure=structure)
    pred_labels, n_lesions_pred = label(pred_data, structure=structure)
    
    tp_count = 0
    fn_count = 0
    detected_volumes = []
    missed_volumes = []
    
    # -- SENSIBILIDAD (Recorrer GT) --
    for i in range(1, n_lesions_gt + 1):
        lesion_mask = (gt_labels == i)
        lesion_vol = np.sum(lesion_mask)
        
        # ¿Toca la predicción esta lesión real?
        overlap = np.sum(pred_data * lesion_mask)
        
        if overlap > 0:
            tp_count += 1
            detected_volumes.append(lesion_vol)
        else:
            fn_count += 1
            missed_volumes.append(lesion_vol)
            
    # -- FALSOS POSITIVOS (Recorrer Predicción) --
    fp_count = 0
    for i in range(1, n_lesions_pred + 1):
        pred_lesion_mask = (pred_labels == i)
        # ¿Toca el GT esta lesión predicha?
        overlap = np.sum(gt_data * pred_lesion_mask)
        
        if overlap == 0:
            fp_count += 1
            
    # --- 3. Métricas Derivadas ---
    if (tp_count + fp_count) > 0:
        precision = tp_count / (tp_count + fp_count)
    else:
        precision = 0.0 if n_lesions_pred > 0 else 1.0
        
    if (tp_count + fn_count) > 0:
        recall = tp_count / (tp_count + fn_count)
    else:
        recall = 1.0 if n_lesions_gt == 0 else 0.0

    if (precision + recall) > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0.0

    return {
        'case_id': case_id,
        'fold': get_cohort(case_id),
        'n_lesions_GT': n_lesions_gt,
        'n_lesions_Pred': n_lesions_pred,
        'TP': tp_count,
        'FP': fp_count,
        'FN': fn_count,
        'Dice_Voxel': round(dice_voxel, 4),
        'detected_vols': str(detected_volumes).replace('[','').replace(']',''),
        'missed_vols': str(missed_volumes).replace('[','').replace(']',''),
        'Lesion_Precision': round(precision, 4),
        'Lesion_Recall': round(recall, 4),
        'F1_Score': round(f1, 4),
        'n_voxels_GT': sum_gt
    }

# --- EJECUCIÓN PRINCIPAL ---
if __name__ == "__main__":
    print(f"Generando métricas (Conectividad 3,2)...")
    print(f" - GT Folder:   {gt_folder}")
    print(f" - Pred Folder: {pred_folder}")
    print(f" - Output CSV:  {output_csv}")

    # Verificación de seguridad
    if not os.path.exists(gt_folder):
        print(f"\nERROR: No encuentro la carpeta de GT: {gt_folder}")
        print("Por favor, edita la variable 'gt_folder' al principio del script.")
        exit()

    results = []
    # Listar archivos basándonos en el GT (que es la referencia)
    files = sorted([f for f in os.listdir(gt_folder) if f.endswith('.nii.gz')])

    for f in files:
        case_id = f.replace('.nii.gz', '')
        
        # Construir rutas usando las variables corregidas
        gt_path = os.path.join(gt_folder, f)
        pred_path = os.path.join(pred_folder, f)
        
        try:
            metrics = compute_metrics(gt_path, pred_path, case_id)
            results.append(metrics)
            print(f"Procesado: {case_id} | TP: {metrics['TP']} | FP: {metrics['FP']} | FN: {metrics['FN']}")
        except Exception as e:
            print(f"Error procesando {case_id}: {e}")

    # Guardar CSV
    if results:
        df = pd.DataFrame(results)
        cols = ['case_id', 'fold', 'n_lesions_GT', 'n_lesions_Pred', 'TP', 'FP', 'FN', 
                'Dice_Voxel', 'detected_vols', 'missed_vols', 
                'Lesion_Precision', 'Lesion_Recall', 'F1_Score', 'n_voxels_GT']
        df = df[cols]

        df.to_csv(output_csv, index=False)
        print(f"\n¡Hecho! Archivo guardado correctamente en:\n{output_csv}")
    else:
        print("\nNo se generaron resultados. Revisa las rutas.")

Generando métricas (Conectividad 3,2)...
 - GT Folder:   C:\Users\marta\Desktop\Predicciones2D3D_Weighted7030_Threshold03\GT
 - Pred Folder: C:\Users\marta\Desktop\Predicciones2D3D_Weighted7030_Threshold03\Predicciones
 - Output CSV:  C:\Users\marta\Desktop\Predicciones2D3D_Weighted7030_Threshold03\Predicciones\Resultados_Predicciones2D3D_Weighted7030_Threshold03.csv
Procesado: sub-105 | TP: 0 | FP: 0 | FN: 0
Procesado: sub-108 | TP: 6 | FP: 0 | FN: 1
Procesado: sub-202 | TP: 0 | FP: 1 | FN: 0
Procesado: sub-215 | TP: 0 | FP: 0 | FN: 1
Procesado: sub-217 | TP: 1 | FP: 0 | FN: 0
Procesado: sub-230 | TP: 0 | FP: 1 | FN: 0
Procesado: sub-232 | TP: 1 | FP: 2 | FN: 0
Procesado: sub-234 | TP: 1 | FP: 1 | FN: 1
Procesado: sub-303 | TP: 1 | FP: 0 | FN: 0
Procesado: sub-308 | TP: 1 | FP: 0 | FN: 0
Procesado: sub-309 | TP: 1 | FP: 0 | FN: 0
Procesado: sub-315 | TP: 1 | FP: 1 | FN: 0
Procesado: sub-316 | TP: 1 | FP: 0 | FN: 1

¡Hecho! Archivo guardado correctamente en:
C:\Users\marta\Desktop\Pred